벡터라이징을 한 뒤 term frequency matrix까지 만드는 부분은 빠르게 넘어가도록 하겠습니다. 

In [29]:
corpus_fname = '../../../data/corpus_10days/news/2016-10-24_article_all_normed.txt'
mm_fname = '../../../data/corpus_10days/models/2016-10-24_article_all_normed_corpus.mtx'
mm_vocab = '../../../data/corpus_10days/models/2016-10-24_article_all_normed_corpus.vocab'
dictionary_fname = '../../../data/corpus_10days/models/2016-10-24_article_all_normed_corpus.dictionary'

ldamodel_fname = '../../../data/corpus_10days/models/2016-10-24_article_all_normed_corpus_lda_3_6.pkl'

TRAIN_LDA = False

import pickle

from sklearn.feature_extraction.text import CountVectorizer
from scipy.io import mmwrite, mmread

In [4]:
from soynlp.utils import DoublespaceLineCorpus
corpus = DoublespaceLineCorpus(corpus_fname, iter_sent=False)
print('num docs =', len(corpus))


with open('2016-10-24-extract_noun_dict.pkl', 'rb') as f:
    noun_dict = pickle.load(f)
    
def custom_tokenize(doc):    
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            if subword in noun_dict:
                return subword
        return ''
    
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns


vectorizer = CountVectorizer(tokenizer=custom_tokenize)
x = vectorizer.fit_transform(corpus)
idx2vocab = [vocab for vocab, idx in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])]

# mmwrite(mm_fname, x)
# with open(mm_vocab, 'w', encoding='utf-8') as f:
#     for word, _ in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1]):
#         f.write('%s\n' % word)

print(x.shape)

num docs = 26368
(26368, 4760)


## Gensim tutorial

### [Corpus Formats][corpus_format]

Gensim의 LDA가 이용하는 학습데이터의 형식은 아래와 같습니다. list of list of tuple이며, tuple은 (term_index, frequency) 입니다.


    from gensim import corpora
    
    # create a toy corpus of 2 documents, as a plain Python list
    corpus = [[(1, 0.5)], []]
    
scikit-learn 을 이용하여 sparse matrix 를 미리 만들었다면 gensim.matutils.Sparse2Corpus 를 이용하여 이를 변형합니다. 그런데, gensim은 sparse matrix를 (doc, term) matrix가 아니라 (term, doc) matrix 라고 가정합니다. 그래서 Sparse2Corpus 의 documents_columns 를 False 로 지정해줍니다. Default 는 True 입니다.

    from scipy.io import mmread
    
    scipy_sparse_matrix = mmread(mm_fname)    
    # corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
    corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)

[corpus_format]: https://radimrehurek.com/gensim/tut1.html#corpus-formats

In [5]:
import scipy
scipy.__version__

'1.1.0'

In [6]:
import gensim

print(gensim.__version__)

3.6.0


In [7]:
corpus = gensim.matutils.Sparse2Corpus(x, documents_columns=False)

for i, doc in enumerate(corpus):
    if i > 3: break
    print('#doc= %d: ' % i, doc[:10], '...')

#doc= 0:  [] ...
#doc= 1:  [(4592, 1), (818, 2), (697, 1), (3202, 1), (4678, 1), (530, 3), (1188, 1), (1605, 1), (3152, 1), (2104, 1)] ...
#doc= 2:  [(2552, 1), (4671, 1), (1174, 1), (3678, 1), (4615, 2), (447, 1), (4740, 1), (1871, 1), (3131, 1), (3679, 1)] ...
#doc= 3:  [(3868, 1), (3513, 1), (2701, 1), (2117, 1), (756, 1), (2858, 1), (1890, 1), (2530, 1), (2189, 1), (469, 1)] ...


In [8]:
len(corpus)

26368

LDA 를 학습할 때에는 corpus 에 등장한 4592 번이 어떤 단어인지 정보를 알아야 합니다. 이를 위해 vectorizer 로부터 id2word 라는 list of str 을 만듭니다.

In [11]:
id2word = {
    idx:vocab for vocab, idx in
    sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
}

이제 데이터를 gensim LDA format으로 맞추는 일이 끝났습니다.

num_topics는 토픽의 개수를 정하는 부분입니다. id2word를 입력하지 않으면 단어가 term index로 출력됩니다. Gensim 의 id2word 는 list of str 이 아닌, dict 구조여야 합니다.

- 학습시간 (i7-5820): 1min 15sec

In [12]:
from gensim.models import LdaModel

if TRAIN_LDA:
    ldamodel = LdaModel(corpus=corpus, num_topics=50, id2word=id2word)
    import pickle
    with open(ldamodel_fname, 'wb') as f:
        pickle.dump(ldamodel, f)
else:
    import pickle
    with open(ldamodel_fname, 'rb') as f:
        ldamodel = pickle.load(f)

print_topic은 특정 topic에 대하여 설명력이 좋은 (topic probability가 높은) topn개의 단어를 prob.와 함께 출력해줍니다. 

In [13]:
ldamodel.print_topic(10, topn=5)

'0.202*"뉴스" + 0.136*"저작권자" + 0.046*"카페" + 0.041*"웹툰" + 0.041*"02"'

get_topic_terms(topic_id)를 하면 term index가 출력되기 때문에 print_topic()의 결과에서 단어만 선택하는 parse_topic_words() 함수를 만들어 50개의 토픽에 대하여 각각 대표단어를 뽑아봅니다. 

In [14]:
ldamodel.get_topic_terms(1)

[(2046, 0.030306516),
 (3660, 0.021786332),
 (2183, 0.021200346),
 (4364, 0.020792965),
 (1955, 0.01804878),
 (905, 0.017082699),
 (4664, 0.015296279),
 (1105, 0.014546601),
 (4453, 0.0141129615),
 (2255, 0.011956487)]

In [15]:
def parse_topic_words(topic_str):
    return [col.split('*"')[1][:-1] for col in topic_str.split(' + ')]

for i in range(50):
    print('#topic= %d:' % i, parse_topic_words( ldamodel.print_topic(i, topn=10)) )
    
    if i % 5 == 4: print()

#topic= 0: ['단지', '아파트', '이용', '예정', '해수', '분양', '지역', '환경', '위치', '골프']
#topic= 1: ['사진', '제품', '선보', '패션', '브랜드', '기사', '화보', '다양', '피부', '세번째']
#topic= 2: ['웃음', '클린턴', '있습니다', '트럼프', '미국', '혼술', '후보', '앵커', '방송', '라고']
#topic= 3: ['기업', '기술', '지원', '사업', '세계', '창업', '개발', '글로벌', '산업', '한국']
#topic= 4: ['영화', '공연', '일간스포츠', '개봉', '고양이', '작품', '할머니', '장애인', '관객', '배우']

#topic= 5: ['개헌', '우리', '국민', '헌법', '사회', '대한민국', '대통령', '지금', '개정', '체제']
#topic= 6: ['제보', '배포', '라이브', '기억', '동영상', '전주', '11월', '훈훈', '자신', '결정']
#topic= 7: ['현대', '인천', '공장', '필리핀', '대사', '이동', '건조', '러시아', '옥녀', '계약']
#topic= 8: ['예산', '정부', '대통령', '국회', '예산안', '지원', '편성', '경제', '시정연설', '관련']
#topic= 9: ['행사', '지난', '참여', '제공', '000원', '진행', '함께', '현지', '부산', '전국']

#topic= 10: ['뉴스', '저작권자', '카페', '웹툰', '02', '이용', '인터넷', '01', '24', '이용자']
#topic= 11: ['문화', '체험', '지역', '제공', '진행', '여행', '운영', '할인', '마련', '다양']
#topic= 12: ['총장', '청춘', '충남', '대학', '한국', '세종역', '신설', '한의사', '아시아', '세종']
#topic= 13: ['남자', '여자',

각 topic 의 단어생성확률은 아래의 함수로 가져올 수 있습니다. LdaModel.expElogbeta 에도 비슷한 정보가 포함되어 있지만, 정확한 정보는 아래의 함수를 이용해야 합니다.

In [16]:
ldamodel.expElogbeta.shape

(50, 4760)

In [17]:
def get_topic_term_prob(lda_model):
    topic_term_freqs = lda_model.state.get_lambda()
    topic_term_prob = topic_term_freqs / topic_term_freqs.sum(axis=1)[:, None]
    return topic_term_prob

In [18]:
beta = get_topic_term_prob(ldamodel)

In [19]:
topn = 20
important_terms = beta.argsort(axis=1)[:,-topn:]
print(important_terms.shape)

(50, 20)


위의 print_topic() 함수의 결과는 아래의 과정을 거쳐 출력된 함수입니다.

In [20]:
for topic_idx in range(beta.shape[0]):
    if topic_idx % 5 == 0:
        print()
    
    term_indices = important_terms[topic_idx,:].reshape(-1)
    terms = reversed([idx2vocab[idx] for idx in term_indices])

    print('topic #{} : {}'.format(topic_idx, ' '.join(terms)))


topic #0 : 단지 아파트 이용 예정 해수 분양 지역 환경 위치 골프 공급 구성 조성 전자신문 멜로디 제공 주거 게스트 가능 지하
topic #1 : 사진 제품 선보 패션 브랜드 기사 화보 다양 피부 세번째 함께 인체 공개 판매 제공 24 가을 아이 음식 스포츠
topic #2 : 웃음 클린턴 있습니다 트럼프 미국 혼술 후보 앵커 방송 라고 유지 그녀 사로 대선 토론 공명 선거 모습 지지 시청자들
topic #3 : 기업 기술 지원 사업 세계 창업 개발 글로벌 산업 한국 성장 경제 시장 투자 계획 연구 교육 전문 미래 분야
topic #4 : 영화 공연 일간스포츠 개봉 고양이 작품 할머니 장애인 관객 배우 티켓 11월 사랑 3일 뮤지컬 진행 감독 예술 주인 코미디

topic #5 : 개헌 우리 국민 헌법 사회 대한민국 대통령 지금 개정 체제 정책 정치 경제 국회 변화 있습니다 논의 1987년 미래 정부
topic #6 : 제보 배포 라이브 기억 동영상 전주 11월 훈훈 자신 결정 기록 저자 법인세 수학 확인 9년 족보 16일 인물들 광복군
topic #7 : 현대 인천 공장 필리핀 대사 이동 건조 러시아 옥녀 계약 24 가짜 평년 호위 순천 처방 함유 수주 중국 불법
topic #8 : 예산 정부 대통령 국회 예산안 지원 편성 경제 시정연설 관련 내년도 연설 확대 박근혜 국민 문화 내년 여야 야당 심사
topic #9 : 행사 지난 참여 제공 000원 진행 함께 현지 부산 전국 마을 개최 주말 지역 광주 이번 학생들 저녁 아스트로 파일

topic #10 : 뉴스 저작권자 카페 웹툰 02 이용 인터넷 01 24 이용자 검색 밴드 포털 독자 마이데일리 시사 블로그 실시간 이메일 이용자들
topic #11 : 문화 체험 지역 제공 진행 여행 운영 할인 마련 다양 이번 참여 실시 예정 주민 있도록 보험 함께 직접 프로그램
topic #12 : 총장 청춘 충남 대학 한국 세종역 신설 한의사 아시아 세종 합의 의장 회장 충북도 중앙 상단 원유 뉴스1 지역 지난

gensim에서 단어의 topic vector를 직접적으로 찾아주는 함수가 구현되어 있지 않습니다. 하지만 LdaModel[bow_model]을 넣으면 bow_model에 대한 topic vector를 출력해줍니다. bow_model에 단어 한 개를 넣으면 해당 단어의 topic vector를 알 수 있습니다. 

bow_model은 [(term id, weight), (term id, weight), ... ] 형식입니다. 

In [21]:
ldamodel[[(0,1)]]

[(35, 0.51)]

단어는 Vectorizer.vocabulary\_를 이용하거나 Dictionary를 이용할 수 있습니다. 

In [22]:
def encode(words):
    bow = [(vectorizer.vocabulary_.get(w, -1), f) for w,f in words]
    bow = [(w, t) for w, t in bow if w != -1]
    return bow

In [23]:
bow_model = encode([('트와이스', 1)])
ldamodel[bow_model]

[(33, 0.51)]

get_document_topics() 함수를 이용하면 minimum_probability, minimum_phi_value를 조절하며 topic vector를 만들 수 있습니다.  

In [26]:
bow_model = encode([('트와이스', 1)])
ldamodel.get_document_topics(bow_model,  minimum_probability=0.01, minimum_phi_value=0.01)

[(33, 0.51)]

In [27]:
bow_model = encode([('무대', 1)])
ldamodel.get_document_topics(bow_model,  minimum_probability=0.01, minimum_phi_value=0.01)

[(23, 0.51)]

새로운 문서에 대해서도 topic probability vector 를 inferring 할 수 있습니다.

In [28]:
bow_model = encode([('트와이스', 1), ('무대', 1)])
ldamodel.get_document_topics(bow_model, minimum_probability=0.01, minimum_phi_value=0.005)

[(23, 0.34), (33, 0.34)]